In [0]:
import pandas as pd
import numpy as np
import keras
# from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
# from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pickle
import time
import tensorflow as tf

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def f1(y_true, y_pred):
    y_pred = np.rint(y_pred)
    tp = sum(y_true*y_pred)
    tn = sum((1-y_true)*(1-y_pred))
    fp = sum((1-y_true)*y_pred)
    fn = sum(y_true*(1-y_pred))

    p = tp / (tp + fp )
    r = tp / (tp + fn )

    f1 = 2*p*r / (p+r)
    return f1

In [0]:
#def f1_loss(y_true, y_pred):
#    
#    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
#    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
#    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
#    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
#
#    p = tp / (tp + fp + K.epsilon())
#    r = tp / (tp + fn + K.epsilon())
#
#    f1 = 2*p*r / (p+r+K.epsilon())
#    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
#    return  1 - K.mean(f1)

In [0]:
start_time = time.time()
y = pd.read_csv("/content/gdrive/My Drive/data/y_bush_vs_others.csv", header= None)
y_bush = y.values.ravel()
y = pd.read_csv("/content/gdrive/My Drive/data/y_williams_vs_others.csv", header= None)
y_williams = y.values.ravel()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.030671358108520508 seconds ---


In [0]:
start_time = time.time()
X = pd.read_csv("/content/gdrive/My Drive/data/X.csv", sep=' ', header= None, dtype= float)
X = X.values
print("--- %s seconds ---" % (time.time() - start_time))

--- 27.466657876968384 seconds ---


In [0]:
X_train, X_test, y_williams_train, y_williams_test = train_test_split(X,y_williams, test_size=1./3, random_state = 6926, shuffle = True, stratify = y_williams)

In [0]:
X_train.shape

(8822, 4096)

In [0]:
X_test.shape

(4411, 4096)

In [0]:
X.shape

(13233, 4096)

In [0]:
X_train = X_train.reshape(8822,64,64,1)

In [0]:
X_test = X_test.reshape(4411,64,64,1)

In [0]:
y_williams_train = y_williams_train.reshape(8822,1)
y_williams_test = y_williams_test.reshape(4411,1)


In [0]:
y_williams_train.shape

(8822, 1)

In [0]:
X_train.shape

(8822, 64, 64, 1)

In [0]:
X_test.shape

(4411, 64, 64, 1)

In [0]:
williams_model = Sequential()

In [0]:
williams_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu',input_shape=(64,64,1)))
# williams_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
williams_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# williams_model.add(Dropout(0.4))

williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
# williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
williams_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# williams_model.add(Dropout(0.4))

# williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
# williams_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
# williams_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

williams_model.add(Flatten())
williams_model.add(Dense(1000,activation='relu'))
williams_model.add(Dense(1,activation='sigmoid'))

In [0]:
williams_model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [0]:
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/williams_training.log')
williams_model.fit(X_train, y_williams_train, batch_size = 1000, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 5s 526us/step - loss: 0.1279 - acc: 0.9904
Epoch 2/10
8822/8822 [==============================] - 1s 165us/step - loss: 0.0619 - acc: 0.9960
Epoch 3/10
8822/8822 [==============================] - 1s 167us/step - loss: 0.0372 - acc: 0.9960
Epoch 4/10
8822/8822 [==============================] - 1s 164us/step - loss: 0.0298 - acc: 0.9960
Epoch 5/10
8822/8822 [==============================] - 1s 162us/step - loss: 0.0262 - acc: 0.9960
Epoch 6/10
8822/8822 [==============================] - 1s 167us/step - loss: 0.0224 - acc: 0.9960
Epoch 7/10
8822/8822 [==============================] - 1s 164us/step - loss: 0.0169 - acc: 0.9960
Epoch 8/10
8822/8822 [==============================] - 1s 165us/step - loss: 0.0122 - acc: 0.9959
Epoch 9/10
8822/8822 [==============================] - 1s 166us/step - loss: 0.0113 - acc: 0.9963
Epoch 10/10
8822/8822 [==============================] - 1s 169us/step - loss: 0.0103 - acc: 0.9964
[0.677966

In [0]:
williams_model.fit(X_train, y_williams_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 2s 241us/step - loss: 0.0089 - acc: 0.9971
Epoch 2/10
8822/8822 [==============================] - 2s 190us/step - loss: 0.0081 - acc: 0.9975
Epoch 3/10
8822/8822 [==============================] - 2s 186us/step - loss: 0.0072 - acc: 0.9976
Epoch 4/10
8822/8822 [==============================] - 2s 183us/step - loss: 0.0052 - acc: 0.9988
Epoch 5/10
8822/8822 [==============================] - 2s 185us/step - loss: 0.0047 - acc: 0.9988
Epoch 6/10
8822/8822 [==============================] - 2s 183us/step - loss: 0.0044 - acc: 0.9986
Epoch 7/10
8822/8822 [==============================] - 2s 184us/step - loss: 0.0026 - acc: 0.9993
Epoch 8/10
8822/8822 [==============================] - 2s 183us/step - loss: 0.0022 - acc: 0.9993
Epoch 9/10
8822/8822 [==============================] - 2s 183us/step - loss: 0.0018 - acc: 0.9995
Epoch 10/10
8822/8822 [==============================] - 2s 187us/step - loss: 0.0020 - acc: 0.9997
[1.]
[0.4

In [0]:
williams_model.fit(X_train, y_williams_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_williams_train,williams_model.predict(X_train)))
print(f1(y_williams_test,williams_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 2s 182us/step - loss: 0.0019 - acc: 0.9993
Epoch 2/10
8822/8822 [==============================] - 2s 183us/step - loss: 4.7440e-04 - acc: 1.0000
Epoch 3/10
8822/8822 [==============================] - 2s 184us/step - loss: 4.9106e-04 - acc: 0.9999
Epoch 4/10
8822/8822 [==============================] - 2s 186us/step - loss: 2.9840e-04 - acc: 1.0000
Epoch 5/10
8822/8822 [==============================] - 2s 182us/step - loss: 1.3916e-04 - acc: 1.0000
Epoch 6/10
8822/8822 [==============================] - 2s 185us/step - loss: 7.8217e-05 - acc: 1.0000
Epoch 7/10
8822/8822 [==============================] - 2s 184us/step - loss: 4.9392e-05 - acc: 1.0000
Epoch 8/10
8822/8822 [==============================] - 2s 183us/step - loss: 3.9160e-05 - acc: 1.0000
Epoch 9/10
8822/8822 [==============================] - 2s 183us/step - loss: 3.2832e-05 - acc: 1.0000
Epoch 10/10
8822/8822 [==============================] - 2s 182us/step - loss

In [0]:
print(f1(y_williams_test,williams_model.predict(X_test)))

[0.53846154]


In [0]:
williams_model.save("/content/gdrive/My Drive/data/williams.model")

In [0]:
pickle.dump([f1(y_williams_train,williams_model.predict(X_train))[0],f1(y_williams_test,williams_model.predict(X_test))[0]], open('/content/gdrive/My Drive/data/williams.pkl', 'wb'))

In [0]:
# pickle.load(open('/content/gdrive/My Drive/data/williams.pkl', 'rb'))